In [1]:
#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter

In [34]:
# Read json into a pandas dataframe
tweets_df = pd.read_json("insurance.txt", lines=True)

In [35]:
len(tweets_df)

1436

In [3]:
import tensorflow as tf 
import tensorflow_hub as hub

print('tf:')
tf.executing_eagerly()

module_url = "use_model"
module_url = 'http://tfhub.dev/google/universal-sentence-encoder-large/5'
use_model = hub.load(module_url)

tf:


In [4]:
import math
def apply_func(x):
    if not isinstance(x, float):
        if 'full_text' in x: 
            return x['full_text']
        else:
            return float('nan')
    else:
        return float('nan')
    

tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
tweets_df.full_text.fillna(tweets_df.text, inplace=True)

In [41]:
tweets_df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,display_text_range,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2020-05-14 21:11:07+00:00,1261041373089869827,1261041373089869824,[🔴 NEWS] How to Rent a Camper https://t.co/eLm...,"<a href=""https://u.fooo.ooo"" rel=""nofollow"">u....",True,NaN,NaN,NaN,NaN,...,low,en,2020-05-14 21:11:07.017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-14 21:11:59+00:00,1261041591231426560,1261041591231426560,Cyber Liability is the risk that is posed by b...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,NaN,NaN,NaN,...,low,en,2020-05-14 21:11:59.026,"[0, 140]",NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-14 21:12:55+00:00,1261041827185954816,1261041827185954816,@TheHartford\n \n@TheHartford_PR\n \n@CDInews\...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,NaN,19614674.0,19614674.0,...,low,en,2020-05-14 21:12:55.282,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-14 21:13:30+00:00,1261041974196539394,1261041974196539392,RT @AWFPAfrica: Join us on Thur 21st May at 4p...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,low,en,2020-05-14 21:13:30.332,NaN,{'created_at': 'Thu May 14 20:00:03 +0000 2020...,NaN,NaN,NaN,NaN,NaN
4,2020-05-14 21:14:01+00:00,1261042103431372801,1261042103431372800,Join us for Episode 3: Insurance of #SAPIndust...,"<a href=""https://www.sprinklr.com"" rel=""nofoll...",True,NaN,NaN,NaN,NaN,...,low,en,2020-05-14 21:14:01.144,"[0, 140]",NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
for e in tweets_df['entities']:
    print(e['hashtags'])

[{'text': 'Insurance', 'indices': [54, 64]}, {'text': 'RV', 'indices': [65, 68]}, {'text': 'RecreationalVehicle', 'indices': [69, 89]}, {'text': 'MinorLeagueBaseball', 'indices': [90, 110]}]
[]
[{'text': 'Discounts', 'indices': [95, 105]}]
[]
[{'text': 'SAPIndustriesLive', 'indices': [36, 54]}]
[]
[{'text': 'EmergingTechnologies', 'indices': [21, 42]}, {'text': 'BigData', 'indices': [69, 77]}, {'text': 'AI', 'indices': [78, 81]}, {'text': 'ML', 'indices': [82, 85]}, {'text': 'IoT', 'indices': [86, 90]}, {'text': 'DigitalTransformation', 'indices': [91, 113]}, {'text': 'FinTech', 'indices': [114, 122]}, {'text': 'HealthTech', 'indices': [123, 134]}]
[{'text': 'homeowners', 'indices': [39, 50]}, {'text': 'insurance', 'indices': [51, 61]}]
[]
[{'text': 'insurance', 'indices': [70, 80]}]
[{'text': 'insurers', 'indices': [29, 38]}, {'text': 'consumers', 'indices': [78, 88]}, {'text': 'COVID19', 'indices': [96, 104]}]
[{'text': 'flood', 'indices': [62, 68]}, {'text': 'insurance', 'indices': 

In [24]:
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION)

import re

#HappyEmoticons
emoticons_happy = {':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}', ':^)', ':-D', ':D', '8-D',
                   '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D', '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P',
                   ':-P', ':P', 'X-P', 'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)', '<3'}


# Sad Emoticons
emoticons_sad = {':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<', ':-[', ':-<', '=\\', '=/',
                 '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c', ':c', ':{', '>:\\', ';('}

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)


def clean_tweets(tweet):
    # after tweepy preprocessing the colon symbol left remain after      #removing mentions
    # tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    # replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)
    # remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    return tweet


def preprocess(tweet):
    preprocessed_tweet = p.clean(tweet)
    return preprocessed_tweet

extracted_texts = tweets_df['full_text'].values
extracted_texts = [clean_tweets(text) for text in extracted_texts]
extracted_texts = [preprocess(text) for text in extracted_texts]

In [25]:
extracted_texts = list(set(extracted_texts))
embeddings = use_model(extracted_texts)

In [26]:
embeddings

<tf.Tensor: shape=(613, 512), dtype=float32, numpy=
array([[-0.02091571, -0.02393747,  0.01697918, ...,  0.0549162 ,
        -0.06758986,  0.01089254],
       [ 0.00525019,  0.00562264,  0.02579736, ...,  0.04216328,
        -0.01819112,  0.0610274 ],
       [ 0.04129343, -0.0483016 ,  0.00920759, ..., -0.01671836,
        -0.01665323,  0.03889812],
       ...,
       [-0.03595069,  0.01082813,  0.01415693, ..., -0.02383216,
        -0.00063798,  0.06291674],
       [-0.04287026,  0.0299869 , -0.01676859, ..., -0.00482234,
        -0.08796454,  0.07278396],
       [ 0.0327877 , -0.02116094, -0.01745663, ...,  0.00451419,
        -0.00355715,  0.06931655]], dtype=float32)>

In [27]:
import hdbscan
from sklearn.datasets import make_blobs



clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
cluster_labels = clusterer.fit_predict(embeddings)

In [28]:
cluster_labels

array([-1, -1, -1, -1, -1, -1, -1, 18, -1, -1, -1, 12, -1,  3,  8, -1, -1,
       -1, -1, -1, -1, -1, 10, -1, -1,  3, -1, -1, -1, -1, -1, -1, -1, 28,
       -1, -1, -1, -1, 21, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 19, 28,
       27, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 17, -1, -1, -1,
       19, -1, -1, -1,  7, -1, -1, 24, -1, 28, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  7, -1, -1, -1, -1, 13, -1,  4, -1, 28, -1, -1,
       -1, -1, -1, -1, 20, 26, 21, -1, -1, -1, -1, -1, -1, -1, 13, -1, -1,
       -1, -1, -1, -1, 28, -1, -1, 10, -1, 21, -1, -1, 21, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, 11, -1, -1,  1, -1, 23, 24, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, 20, -1, 21, -1, 10, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, 15, -1, -1, -1, -1, -1, -1, 21, -1,
       -1,  2, -1, 20,  9, -1, -1, -1,  8, -1, -1,  5, -1, -1, -1, -1, 21,
       -1, -1, -1, 21, -1, -1, 26, -1, 27, -1, -1, -1, -1, -1, 28, 25, -1,
       -1, -1, -1, -1, -1

In [29]:
max(cluster_labels)

29

In [30]:
cluster_list = [[] for i in range(max(cluster_labels) + 1)]

In [31]:
tweet_cluster_dict = {}
for tweet_index, cluster_index in enumerate(cluster_labels):
    if cluster_index != -1:
        cluster_list[cluster_index].append(tweet_index)
        
        

In [32]:
for cluster in cluster_list:
    print('cluster::::::', cluster)
    for tweet_id in cluster:
        print(extracted_texts[tweet_id])

cluster:::::: [364, 397, 580]
RT : #Article from sheds light on the reality of mental health in the #legal profession. Learn more:
Via #Reuters, Advocacy group says TikTok violated FTC consent decree and children's privacy rules #LegalTech #LegalNews #LegalServices #Lawyers #Attorneys #Legal #Law (#Lawfirm #Attorney #Lawyer)
RT : Friday's LawNewsIndex: 15th May - Law News #Lawyers #Lawfirms #Legal #HumanRights #Legislation #Just
cluster:::::: [145, 490, 509, 596]
RT : Letter C Logo Stylish for sale #Modern #simple #unique #letter #C #young #sleek #technology #cool #s
RT : Letter K Logo Technology for Sale #Modern #simple #unique #techie #letter #K #young #elegant #active
RT : Letter C Logo Sporty for Sale #Modern #simple #unique #adventure #sleek #cool #serious #solid #corpo
RT : Lion Logo Technology for sale #Modern #simple #unique #head #face #professional #technology #softwar
cluster:::::: [188, 282, 325, 353]
RT : #420friendly #weedlover Cannabis May Stop Coronavirus From Infecting